In [1]:
import pandas as pd

In [40]:
data = {'label_reference': [1, 2, 3, 4, 5, 6], 'area_reference' : [400, 900, 1400, 900, 1100, 1800]}
table_ref = pd.DataFrame(data)
data2 = {'label_reference': [0, 1, 2, 3, 4, 5, 6, 6], 'label_ch1': [8, 7, 6, 5, 4, 3, 1, 2]}
table_linking_1 = pd.DataFrame(data2)
data3 = {'label_reference': [0, 1, 2, 3, 4, 5, 6, 6, 6], 'label_ch2': [7, 0, 6, 5, 4, 3, 1, 2, 8]}
table_linking_2 = pd.DataFrame(data3)

table_ch1 = pd.DataFrame(data={'label_ch1': [1, 2, 3, 4, 5, 6, 7, 8], 'area_ch1' : [40, 90, 140, 90, 110, 180, 20, 100]})
table_ch2 = pd.DataFrame(data={'label_ch2': [1, 2, 3, 4, 5, 6, 7, 8], 'area_ch2' : [40, 90, 140, 90, 110, 180, 20, 100]})

output tables must be separated in a list, otherwise statistics like counts and std will be wrong (table would have duplicated values).

Below, I am appending each table in a list instead of merging them as before and having this problem.

If I do so, then measure_labels_in_labels will have to return a list because merge_measurements_to_reference will return a list if it gets a list of labels_to_measure..
Then, jupyter notebook explaining functions must be updated..

But make_summary_table must return a single table, so that napari gets only one table output.
I am trying to make it work in this notebook below.


In [41]:
## output_table starts with reference labels and their properties
output_table = table_ref
n_measurement_tables = 2
list_table_linking_labels = [table_linking_1, table_linking_2]
list_table_labels_to_measure_properties = [table_ch1, table_ch2]
suffixes = ['_reference', '_ch1', '_ch2']
output_table_list = []
## Consecutively merge linking_labels tables and properties from other channels tables to the reference table
for i, table_linking_labels, table_labels_to_measure_properties in zip(range(n_measurement_tables), list_table_linking_labels, list_table_labels_to_measure_properties):
    ## !!!Problem!!!: If I do like this, with more than 2 tables, I am also duplicating the second labels
    # Merge other labels to output table based on label_reference
    output_table = pd.merge(table_ref,
                            table_linking_labels,
                            how='outer', on='label' + suffixes[0])
    # Fill NaN labels with zeros (if label were not linked, they belong to background)
    output_table['label' + suffixes[i+1]] = output_table['label' + suffixes[i+1]].fillna(0)
#     # Merge other properties to output table based on new labels column
    output_table = pd.merge(output_table,
                            table_labels_to_measure_properties,
                            how='outer', on='label' + suffixes[i+1])
    
    print(i)
    # Ensure label columns type to be integer
    for column in output_table.columns:
        if column.startswith('label'):
            output_table[column] = output_table[column].astype(int)
    
    output_table_list.append(output_table)


0
1


In [42]:
output_table_list[0]

,label_reference,area_reference,label_ch1,area_ch1
0,1,400.0,7,20
1,2,900.0,6,180
2,3,1400.0,5,110
3,4,900.0,4,90
4,5,1100.0,3,140
5,6,1800.0,1,40
6,6,1800.0,2,90
7,0,NaN,8,100


In [43]:
output_table_list[1]

,label_reference,area_reference,label_ch2,area_ch2
0,1,400.0,0,NaN
1,2,900.0,6,180.0
2,3,1400.0,5,110.0
3,4,900.0,4,90.0
4,5,1100.0,3,140.0
5,6,1800.0,1,40.0
6,6,1800.0,2,90.0
7,6,1800.0,8,100.0
8,0,NaN,7,20.0


In [114]:
# Now table will be a list
table = output_table_list
suffixes = None
statistics_list = ['count', 'mean']
import re

In [115]:
if suffixes is None:
    suffixes = []
    pattern = 'label*(_\w+)$' # get everything after '_' that starts with 'label'
    for tab in table:
        for name in tab.columns:
            matches = re.match(pattern, name)
            if matches is not None:
                new_entry = matches.group(1)
                if new_entry not in suffixes:
                    suffixes.append(new_entry)
        if len(suffixes) == 0:
            print('Could not infer suffixes from column names. Please provide a list of suffixes identifying different channels')
            

In [116]:
suffixes

['_reference', '_ch1', '_ch2']

In [117]:
table[1]

,label_reference,area_reference,label_ch2,area_ch2
0,1,400.0,0,NaN
1,2,900.0,6,180.0
2,3,1400.0,5,110.0
3,4,900.0,4,90.0
4,5,1100.0,3,140.0
5,6,1800.0,1,40.0
6,6,1800.0,2,90.0
7,6,1800.0,8,100.0
8,0,NaN,7,20.0


In [118]:
len(suffixes[1:])

2

In [119]:
if 'count' in statistics_list:
    counts = True
    statistics_list.remove('count')
else:
    counts = False
summary_table_list = []
for tab, suf in zip(table, suffixes[1:]):

    grouped = tab.groupby('label' + suffixes[0])
    probe_columns = [prop for prop in tab.columns
                        if not prop.endswith(suffixes[0])]
    probe_measurement_columns = [name for name in probe_columns
                        if not name.startswith('label')]
    summary_tab = grouped[probe_measurement_columns].describe().reset_index()


    

    # Filter by selected statistics
    selected_columns = [('label' + suffixes[0], '')]
    for stat in statistics_list:
        for column in summary_tab.columns:

            column_stat = column[-1]
            if stat == column_stat:
                selected_columns.append(column)
    summary_tab = summary_tab.loc[:,selected_columns]
    
    if counts:
        # for suf in suffixes[1:]:
        print(tab)
        # counts [label + suf] elements grouped by label_reference
        counts_column = tab.groupby('label' + suffixes[0]).count()['label' + suf].fillna(0)
        # if only 'counts' was asked, append to table
        if len(statistics_list)==0:
            summary_tab['counts' + suf] = counts_column
        # otherwise, insert 'counts' at column just before each suffix features
        else:
            for i, column in enumerate(summary_tab.columns):
                if (column[0].endswith(suf)):
                    summary_tab.insert(i, 'counts' + suf, counts_column)
                    break
    
    summary_table_list.append(summary_tab)


   label_reference  area_reference  label_ch1  area_ch1
0                1           400.0          7        20
1                2           900.0          6       180
2                3          1400.0          5       110
3                4           900.0          4        90
4                5          1100.0          3       140
5                6          1800.0          1        40
6                6          1800.0          2        90
7                0             NaN          8       100
   label_reference  area_reference  label_ch2  area_ch2
0                1           400.0          0       NaN
1                2           900.0          6     180.0
2                3          1400.0          5     110.0
3                4           900.0          4      90.0
4                5          1100.0          3     140.0
5                6          1800.0          1      40.0
6                6          1800.0          2      90.0
7                6          1800.0          8   

In [120]:
len(summary_table_list)

2

In [121]:
summary_table = summary_table_list[0]
for summary_tab in summary_table_list[1:]:
    summary_table = pd.concat([summary_table, summary_tab.iloc[:,1:]], axis=1)
summary_table.columns = [' '.join(col).strip()
                     for col in summary_table.columns.values]
summary_table

,label_reference,counts_ch1,area_ch1 mean,counts_ch2,area_ch2 mean
0,0,1,100.0,1,20.000000
1,1,1,20.0,1,NaN
2,2,1,180.0,1,180.000000
3,3,1,110.0,1,110.000000
4,4,1,90.0,1,90.000000
5,5,1,140.0,1,140.000000
6,6,2,65.0,3,76.666667


test for table 0

In [47]:
# for tab in table:
tab = table[0]
grouped = tab.groupby('label' + suffixes[0])
probe_columns = [prop for prop in tab.columns
                    if not prop.endswith(suffixes[0])]
probe_measurement_columns = [name for name in probe_columns
                    if not name.startswith('label')]
summary_table = grouped[probe_measurement_columns].describe().reset_index()
print(summary_table)

if 'count' in statistics_list:
    counts = True
    statistics_list.remove('count')
else:
    counts = False

# Filter by selected statistics
selected_columns = [('label' + suffixes[0], '')]
for stat in statistics_list:
    for column in summary_table.columns:

        column_stat = column[-1]
        if stat == column_stat:
            selected_columns.append(column)
summary_table = summary_table.loc[:,selected_columns]
summary_table1 = summary_table
summary_table1

  label_reference area_ch1                                                \
                     count   mean        std    min    25%    50%    75%   
0               0      1.0  100.0        NaN  100.0  100.0  100.0  100.0   
1               1      1.0   20.0        NaN   20.0   20.0   20.0   20.0   
2               2      1.0  180.0        NaN  180.0  180.0  180.0  180.0   
3               3      1.0  110.0        NaN  110.0  110.0  110.0  110.0   
4               4      1.0   90.0        NaN   90.0   90.0   90.0   90.0   
5               5      1.0  140.0        NaN  140.0  140.0  140.0  140.0   
6               6      2.0   65.0  35.355339   40.0   52.5   65.0   77.5   

          
     max  
0  100.0  
1   20.0  
2  180.0  
3  110.0  
4   90.0  
5  140.0  
6   90.0  


,label_reference,area_ch1
,,mean
0,0,100.0
1,1,20.0
2,2,180.0
3,3,110.0
4,4,90.0
5,5,140.0
6,6,65.0


test for table 1

In [48]:
tab = table[1]
grouped = tab.groupby('label' + suffixes[0])
probe_columns = [prop for prop in tab.columns
                    if not prop.endswith(suffixes[0])]
probe_measurement_columns = [name for name in probe_columns
                    if not name.startswith('label')]
summary_table = grouped[probe_measurement_columns].describe().reset_index()
print(summary_table)

if 'count' in statistics_list:
    counts = True
    statistics_list.remove('count')
else:
    counts = False

# Filter by selected statistics
selected_columns = [('label' + suffixes[0], '')]
for stat in statistics_list:
    for column in summary_table.columns:

        column_stat = column[-1]
        if stat == column_stat:
            selected_columns.append(column)
summary_table = summary_table.loc[:,selected_columns]
summary_table2 = summary_table
summary_table2

  label_reference area_ch2                                                     \
                     count        mean        std    min    25%    50%    75%   
0               0      1.0   20.000000        NaN   20.0   20.0   20.0   20.0   
1               1      0.0         NaN        NaN    NaN    NaN    NaN    NaN   
2               2      1.0  180.000000        NaN  180.0  180.0  180.0  180.0   
3               3      1.0  110.000000        NaN  110.0  110.0  110.0  110.0   
4               4      1.0   90.000000        NaN   90.0   90.0   90.0   90.0   
5               5      1.0  140.000000        NaN  140.0  140.0  140.0  140.0   
6               6      3.0   76.666667  32.145503   40.0   65.0   90.0   95.0   

          
     max  
0   20.0  
1    NaN  
2  180.0  
3  110.0  
4   90.0  
5  140.0  
6  100.0  


,label_reference,area_ch2
,,mean
0,0,20.000000
1,1,NaN
2,2,180.000000
3,3,110.000000
4,4,90.000000
5,5,140.000000
6,6,76.666667


In [63]:
summary_table_list = [summary_table1, summary_table2]

In [67]:
final_summary_table = summary_table1
for summary in summary_table_list[1:]:
    final_summary_table = pd.concat([final_summary_table, summary.iloc[:,1:]], axis=1)
final_summary_table.columns = [' '.join(col).strip()
                     for col in final_summary_table.columns.values]
final_summary_table

,label_reference,area_ch1 mean,area_ch2 mean
0,0,100.0,20.000000
1,1,20.0,NaN
2,2,180.0,180.000000
3,3,110.0,110.000000
4,4,90.0,90.000000
5,5,140.0,140.000000
6,6,65.0,76.666667


In [ ]:
def make_summary_table(table: List["pandas.DataFrame"],
                      suffixes=None,
                      statistics_list = ['count',]) -> "pandas.DataFrame":
    # If not provided, guess suffixes from column names (last string after '_')
    import re
    import pandas as pd
    if suffixes is None:
        suffixes = []
        pattern = 'label*(_\w+)$' # get everything after '_' that starts with 'label'
        for tab in table:
            for name in tab.columns:
                matches = re.match(pattern, name)
                if matches is not None:
                    new_entry = matches.group(1)
                    if new_entry not in suffixes:
                        suffixes.append(new_entry)
            if len(suffixes) == 0:
                print('Could not infer suffixes from column names. Please provide a list of suffixes identifying different channels')

    if 'count' in statistics_list:
        counts = True
        statistics_list.remove('count')
    else:
        counts = False
    
    summary_table_list = []
    for tab, suf in zip(table, suffixes[1:]):

        grouped = tab.groupby('label' + suffixes[0])
        probe_columns = [prop for prop in tab.columns
                            if not prop.endswith(suffixes[0])]
        probe_measurement_columns = [name for name in probe_columns
                            if not name.startswith('label')]
        summary_tab = grouped[probe_measurement_columns].describe().reset_index()

        # Filter by selected statistics
        selected_columns = [('label' + suffixes[0], '')]
        for stat in statistics_list:
            for column in summary_tab.columns:

                column_stat = column[-1]
                if stat == column_stat:
                    selected_columns.append(column)
        summary_tab = summary_tab.loc[:,selected_columns]
        
        if counts:
            # counts [label + suf] elements grouped by label_reference
            counts_column = tab.groupby('label' + suffixes[0]).count()['label' + suf].fillna(0)
            # if only 'counts' was asked, append to table
            if len(statistics_list)==0:
                summary_tab['counts' + suf] = counts_column
            # otherwise, insert 'counts' at column just before each suffix features
            else:
                for i, column in enumerate(summary_tab.columns):
                    if (column[0].endswith(suf)):
                        summary_tab.insert(i, 'counts' + suf, counts_column)
                        break
        
        summary_table_list.append(summary_tab)

    # Join summary tables
    summary_table = summary_table_list[0]
    for summary_tab in summary_table_list[1:]:
        summary_table = pd.concat([summary_table, summary_tab.iloc[:,1:]], axis=1)
    # Flatten summary statistics table
    summary_table.columns = [' '.join(col).strip()
                        for col in summary_table.columns.values]
    return summary_table